# 00-Connect database

In [ ]:
import sqlite3
import pandas as pd
from pathlib import Path

SQL_OUT = Path("../sql_outputs/takeaway/")
SQL_OUT.mkdir(parents=True, exist_ok=True)

conn = sqlite3.connect("../databases/takeaway.db")

In [ ]:
# Checking DB Connection is in place
# pd.read_sql_query("SELECT 1 AS ok", conn)

# 01-Price distribution of menu items

In [ ]:
sql = """
SELECT price, COUNT(*) 
FROM menuitems 
GROUP BY price 
ORDER BY price
"""
df_result = pd.read_sql_query(sql, conn)

df_result.to_csv(
    SQL_OUT / "01_price_distribution.csv",
    index=False
)

df_result

# 02-Resto distribution per location

In [ ]:
sql = """
SELECT city, COUNT(*) as num_restaurants 
FROM restaurants 
GROUP BY city
"""
df_result = pd.read_sql_query(sql, conn)

df_result.to_csv(
    SQL_OUT / "02_resto_distribution_by_location.csv",
    index=False
)

df_result

# 03-Top 10 rated pizza

In [ ]:
sql = """
SELECT distinct(r.name), r.ratings
from restaurants as r
join categories as c
on r.primarySlug = c.restaurant_id
where c.name like '%Pizza%'
order by r.ratings DESC
limit 10
"""
df_result = pd.read_sql_query(sql, conn)

df_result.to_csv(
    SQL_OUT / "03_top_10_rated_pizza.csv",
    index=False
)

df_result

# 04-Map locations offering kapsalons and their average price

In [ ]:
sql = """
SELECT 
    r.name AS restaurant_name,
    r.latitude,
    r.longitude,
    AVG(m.price) AS avg_price,
    r.ratings
FROM restaurants AS r
JOIN menuitems AS m
    ON r.primarySlug = m.primarySlug
WHERE LOWER(m.name) LIKE '%kapsalon%'
GROUP BY r.name, r.latitude, r.longitude, r.ratings
ORDER BY avg_price DESC
"""

df_result = pd.read_sql_query(sql, conn)

# Save to CSV
df_result.to_csv(
    SQL_OUT / "04_kapsalon_location_avg_price.csv",
    index=False
)

df_result


# 05-Which restaurants have the best price-to-rating ratio?

In [ ]:
sql = """
SELECT 
    r.name,
    r.ratings,
    AVG(m.price) AS avg_price,
    AVG(m.price) / r.ratings AS price_to_rating
FROM restaurants AS r
JOIN menuitems AS m
    ON r.primarySlug = m.primarySlug
WHERE r.ratings > 0
GROUP BY r.name, r.ratings
ORDER BY price_to_rating
"""
df_result = pd.read_sql_query(sql, conn)

df_result.to_csv(
    SQL_OUT / "05_best_price_to_rating_ratio.csv",
    index=False
)

df_result

# 06-Where are the delivery ‘dead zones’—areas with minimal restaurant coverage?
Just select all restos wtih locations now and use visualization later to identify the delivery dead zones

In [ ]:
sql = """
SELECT 
    l.city,
    l.latitude,
    l.longitude,
    COUNT(ltr.restaurant_id) AS restaurant_count
FROM locations AS l
LEFT JOIN locations_to_restaurants AS ltr
    ON l.ID = ltr.location_id
GROUP BY l.city, l.latitude, l.longitude
ORDER BY restaurant_count ASC
"""
df_result = pd.read_sql_query(sql, conn)

df_result.to_csv(
    SQL_OUT / "06_delivery_dead_zones.csv",
    index=False
)

df_result

# 07-How does the availability of vegetarian and vegan dishes vary by area?

In [ ]:
sql = """
SELECT
    r.name,
    r.latitude,
    r.longitude,
    SUM(c.name LIKE '%Vegetarian%') AS has_vegetarian,
    SUM(c.name LIKE '%Vegan%') AS has_vegan
FROM restaurants AS r
JOIN categories AS c
    ON r.primarySlug = c.restaurant_id
GROUP BY r.name, r.latitude, r.longitude
HAVING SUM(c.name LIKE '%Vegetarian%') + SUM(c.name LIKE '%Vegan%') > 0
"""
df_result = pd.read_sql_query(sql, conn)

df_result.to_csv(
    SQL_OUT / "07_veggie_vegan_by_area.csv",
    index=False
)

df_result

# 08-Identify the World Hummus Order (WHO): top 3 hummus serving restaurants

In [ ]:
sql = """
SELECT DISTINCT
    r.name,
    r.ratings
FROM restaurants AS r
JOIN menuitems AS m
    ON r.primarySlug = m.primarySlug
WHERE m.name LIKE '%Hummus%'
ORDER BY r.ratings DESC
LIMIT 3
"""
df_result = pd.read_sql_query(sql, conn)

df_result.to_csv(
    SQL_OUT / "08_WHO_top_3_hummus.csv",
    index=False
)

df_result

# 09-Which restaurants in Leuven provide best balance between rating and price?

In [ ]:
sql = """
SELECT 
    r.name,
    r.address,
    AVG(m.price) AS avg_price,
    r.ratings,
    AVG(m.price) / r.ratings AS price_to_rating
FROM restaurants AS r
JOIN menuitems AS m
    ON r.primarySlug = m.primarySlug
WHERE r.city = 'Leuven'
GROUP BY r.name, r.ratings
HAVING r.ratings > 0
ORDER BY price_to_rating
LIMIT 20
"""
df_result = pd.read_sql_query(sql, conn)

df_result.to_csv(
    SQL_OUT / "09_best_Leuven_price_rating.csv",
    index=False
)

df_result

# 10-Any restaurants with explicit keto or low carb menu items?

In [ ]:
sql = """
SELECT DISTINCT
    r.name AS restaurant_name,
    r.city
FROM menuitems AS m
JOIN restaurants AS r
    ON r.primarySlug = m.primarySlug
WHERE m.name LIKE '%keto%'
   OR m.name LIKE '%low carb%'
   OR m.description LIKE '%keto%'
   OR m.description LIKE '%low carb%'
"""
df_result = pd.read_sql_query(sql, conn)

df_result.to_csv(
    SQL_OUT / "10_restaurants_with_low_carb_options.csv",
    index=False
)

df_result

# 11-TRYOUTS - locations & locations_to_restaurants used for dead zones iso city of restaurants...

In [ ]:
sql = """
SELECT 
    l.city,
    l.latitude,
    l.longitude,
    COUNT(ltr.restaurant_id) AS restaurant_count
FROM locations AS l
LEFT JOIN locations_to_restaurants AS ltr
    ON l.ID = ltr.location_id
GROUP BY l.city, l.latitude, l.longitude
ORDER BY restaurant_count ASC
"""
df_result = pd.read_sql_query(sql, conn)

df_result